In [226]:
# Library imports
import geopandas as gpd
import osmnx as ox
import networkx as nx
import momepy
import matplotlib.pyplot as plt
import folium
import pandas as pd
import overpy
from shapely.geometry import LineString
import requests


In [227]:
# update osmnx settings
useful_tags_ways = ox.settings.useful_tags_way + ['cycleway'] + ['bicycle'] + ['motor_vehicle'] + ['railway'] + ['tunnel']
ox.config(use_cache=True, 
          log_console=True,
          useful_tags_way=useful_tags_ways
          )

C:\Users\b8008458\AppData\Local\Temp\ipykernel_13020\3787230940.py:3: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True,


In [228]:
# read in UK road gpkg
#os_open_roads = gpd.read_file(r"C:\Users\b8008458\OneDrive - Newcastle University\2022 to 2023\PhD\ltnDetection\LTN-Detection\data\oproad_gpkg_gb\Data\oproad_roads_only.gpkg")

In [229]:
# read in UK rivers gpkg
#os_open_rivers = gpd.read_file(r"C:\Users\b8008458\OneDrive - Newcastle University\2022 to 2023\PhD\ltnDetection\LTN-Detection\data\oproad_gpkg_gb\Data\oprvrs_watercourse_only.gpkg")

In [230]:
# set location and get boundary
place = "Newcastle Upon Tyne, United Kingdom"
boundary = ox.geocode_to_gdf(place)
boundary = boundary.to_crs('EPSG:27700')

# buffer boundary to ensure clips include riverlines which may act as borders between geographies
boundary_buffered = boundary.buffer(50)

In [231]:
## get railways

# for unknown reasons, using rail = ox.graph_from_place(place, custom_filter='["railway"]')
# doesn't ALWAYS retrive the full rail network, hence why multiple lines are used to achive the same result

# Get the major rail network
railway_types = ["","rail", "light_rail", "narrow_gauge", "subway", "tram"]

# set an empty
combined_railways = nx.MultiDiGraph()

for railway_type in railway_types:
    try:
        network = ox.graph_from_place(place, simplify=False 
                                      , custom_filter=f'["railway"~"{railway_type}"]'
                                      )
    
    # handle locations where not all rail types are found
    except Exception as e:
        print(f"No railway data found for '{railway_type}'.")
        network = nx.MultiGraph()

    combined_railways = nx.compose(combined_railways, network)
    


# convert to gdf
railways = ox.graph_to_gdfs(combined_railways, nodes=False, edges=True)

# Drop any tunnels 
railways = railways.loc[railways["tunnel"] != "yes"]


# Drop any other railway types that aren't needed
railways = railways.loc[(~railways["railway"].isin(["abandoned", "razed", "disused", "funicular", "monorail", "miniature"]))]

# set railways crs
railways = railways.to_crs('EPSG:27700')

No railway data found for 'narrow_gauge'.
No railway data found for 'subway'.
No railway data found for 'tram'.


In [232]:
## get unsuitable landcover types

# reset boundary crs to allow for features to be found
boundary_buffered = boundary_buffered.to_crs('EPSG:4326')

# Define tags
tags = {"landuse": ["industrial", "railway", "brownfield", "commercial", "farmland", "meadow"]}

# Use ox.features_from_polygon to find features matching the specified tags
landuse = ox.features_from_polygon(polygon = boundary_buffered.iloc[0], tags = tags)

# set/reset crs
landuse = landuse.to_crs('27700')

## get unsuitable "nature" types

# Define tags
tags = {"natural": ["wood", "water", "scrub"]}

# Use ox.features_from_polygon to find features matching the specified tags
nature = ox.features_from_polygon(polygon = boundary_buffered.iloc[0], tags = tags)

# set/reset crs
nature = nature.to_crs('27700')

## get unsuitable "lesiure" types. This is mainly for golfcourses

# Define tags
tags = {"leisure": ["golf_course", "track"]}

# Use ox.features_from_polygon to find features matching the specified tags
leisure = ox.features_from_polygon(polygon = boundary_buffered.iloc[0], tags = tags)

# set/reset crs
leisure = leisure.to_crs('27700')
boundary_buffered = boundary_buffered.to_crs('27700')

# concat
landuse = pd.concat([landuse, nature, leisure])



In [233]:
## get bus routes from OSM/NAPTAN

# reset boundary crs to allow for features to be found
boundary_buffered = boundary_buffered.to_crs('EPSG:4326')

# Calculate the bounding box for XML query
bounding_box = boundary_buffered.bounds

# Extract the minimum and maximum coordinates
minx = bounding_box['minx'].min()
miny = bounding_box['miny'].min()
maxx = bounding_box['maxx'].max()
maxy = bounding_box['maxy'].max()

# Create a list of four elements representing the bounding box
bbox = [minx, miny, maxx, maxy]

# reset boundary_buffer crs
boundary_buffered = boundary_buffered.to_crs('27700')

# Define the Overpass API endpoint
overpass_url = "https://overpass-api.de/api/interpreter"

# Define the XML query
xml_query = f"""
<osm-script output="json" output-config="" timeout="25">
  <query into="searchArea" type="area">
    <bbox-query s="{bbox[1]}" w="{bbox[0]}" n="{bbox[3]}" e="{bbox[2]}" />
  </query>
  <union into="_">
    <query into="_" type="node">
      <has-kv k="route" modv="" v="bus"/>
      <area-query from="searchArea"/>
    </query>
    <query into="_" type="way">
      <has-kv k="route" modv="" v="bus"/>
      <area-query from="searchArea"/>
    </query>
    <query into="_" type="relation">
      <has-kv k="route" modv="" v="bus"/>
      <area-query from="searchArea"/>
    </query>
  </union>
  <print e="" from="_" geometry="full" ids="yes" limit="" mode="body" n="" order="id" s="" w=""/>
  <recurse from="_" into="_" type="down"/>
  <print e="" from="_" geometry="full" ids="yes" limit="" mode="skeleton" n="" order="quadtile" s="" w=""/>
</osm-script>
"""


# Make the Overpass API request
response = requests.post(overpass_url, data=xml_query)

# Check if the request was successful
#if response.status_code == 200:
#    data = response.json()
    
    # Access the data from the response
#    for element in data.get("elements", []):
#        print(f"Element ID: {element.get('id')}, Type: {element.get('type')}, Tags: {element.get('tags')}")
#else:
#    print(f"Error fetching data: {response.status_code} - {response.text}")

# Initialize lists to store data
geometries = []
element_data = []

# Iterate through the elements in the data
for element in data.get("elements", []):
    if element.get('type') == 'way' and 'geometry' in element:
        # Extract geometry coordinates from 'geometry' field
        coordinates = [(node['lon'], node['lat']) for node in element['geometry']]
        # Create a LineString geometry
        line = LineString(coordinates)
        geometries.append(line)
        element_data.append(element)

# Create a GeoDataFrame
bus_routes = gpd.GeoDataFrame(element_data, geometry=geometries)

# Set CRS
bus_routes = bus_routes.set_crs('4326')
bus_routes = bus_routes.to_crs('27700')

In [234]:
## clip roads, rivers and railways to boundary

# clip
os_open_roads_clip = gpd.clip(os_open_roads, boundary_buffered)
os_open_rivers_clip = gpd.clip(os_open_rivers, boundary_buffered)
railways_clip = gpd.clip(railways, boundary_buffered)
landuse_clip = gpd.clip(landuse, boundary_buffered)
bus_routes_clip = gpd.clip(bus_routes, boundary_buffered)

In [235]:
# Find "neighbourhood" roads
neighbourhoods = os_open_roads_clip.loc[~((os_open_roads_clip['primary_route'] == 'True') |
                        (os_open_roads_clip['trunk_road'] == 'True') |
                        (os_open_roads_clip['fictitious'] == 'True') |
                        (os_open_roads_clip['road_classification'] == 'A Road') | 
                        (os_open_roads_clip['road_classification'] == 'B Road') | 
                        (os_open_roads_clip['road_function'] == 'Restricted Local Access Road') |
                        (os_open_roads_clip['road_function'] == 'Minor Road') |
                        (os_open_roads_clip['road_function'] == 'Motorway') |
                        (os_open_roads_clip['road_function'] == 'Minor Road') 
                        )]

In [236]:
# Find "boundary" roads
boundary_roads = os_open_roads_clip.loc[((os_open_roads_clip['primary_route'] == 'True') |
                        (os_open_roads_clip['trunk_road'] == 'True') |
                        (os_open_roads_clip['fictitious'] == 'True') |
                        (os_open_roads_clip['road_classification'] == 'A Road') | 
                        (os_open_roads_clip['road_classification'] == 'B Road') | 
                        #(os_open_roads_clip['road_function'] == 'Restricted Local Access Road') |
                        (os_open_roads_clip['road_function'] == 'Minor Road') |
                        (os_open_roads_clip['road_function'] == 'Motorway') |
                        (os_open_roads_clip['road_function'] == 'Minor Road') 
                        )]

In [237]:
## buffering and dissolving functions
 
def buffer_and_dissolve(input_gdf):
    # Buffer around boundaries
    buffered_gdf = input_gdf.copy()  # Create a copy to avoid modifying the original
    buffered_gdf['geometry'] = buffered_gdf['geometry'].buffer(5) # set a 5 meter buffer

    # Dissolve the geometries
    dissolved_geo = buffered_gdf.unary_union

    # Create a new GeoDataFrame with the dissolved geometry
    dissolved_gdf = gpd.GeoDataFrame(geometry=[dissolved_geo])

    # Set the CRS (Coordinate Reference System)
    dissolved_gdf.crs = input_gdf.crs

    return dissolved_gdf

def dissolve_gdf(input_gdf):
    # dissolve geometries
    dissolved_geo = input_gdf.unary_union
    dissolved_gdf = gpd.GeoDataFrame(geometry=[dissolved_geo])
    dissolved_gdf.crs = input_gdf.crs

    return dissolved_gdf

In [238]:
# buffer and dissolve 

boundary_roads = buffer_and_dissolve(boundary_roads)
boundary_rivers = buffer_and_dissolve(os_open_rivers_clip)
boundary_rail = buffer_and_dissolve(railways_clip)
boundary_landuse = buffer_and_dissolve(landuse_clip)
boundary_bus_routes = buffer_and_dissolve(bus_routes_clip)

In [245]:
# join all boundary features

boundaries = pd.concat([boundary_rivers 
                        ,boundary_roads 
                        ,boundary_rail 
                        #,boundary_landuse
                        ,boundary_bus_routes
                        ], ignore_index=True)
boundary_features = dissolve_gdf(boundaries)

In [246]:
# Use the `difference` method to perform the "Erase" operation
erased_boundary = boundary.difference(boundary_features.unary_union)

# Convert the GeoSeries to a single geometry using unary_union
erased_boundary = erased_boundary.unary_union

# Create a new GeoDataFrame with the result of "Erase" operation
erased_boundary_gdf = gpd.GeoDataFrame(geometry=[erased_boundary], crs=boundary.crs)

# explode multipolygon to polygons
erased_boundary_gdf = erased_boundary_gdf.explode()


C:\Users\b8008458\AppData\Local\Temp\ipykernel_13020\94038513.py:11: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  erased_boundary_gdf = erased_boundary_gdf.explode()


In [247]:
neighbourhoods = erased_boundary_gdf

In [248]:
## drop very small areas (such as the centre of roundabouts etc)

# calculate area
neighbourhoods["area"] = neighbourhoods.geometry.area
# Drop rows where area is less than 5000. This value is arbitary
neighbourhoods = neighbourhoods.loc[neighbourhoods["area"] >= 10000]

In [249]:
## drop areas with no roads

import geopandas as gpd

def count_roads_within_polygons(polygons_gdf, roads_gdf, polygon_column_name):
    """
    Count the number of roads within each polygon in a GeoDataFrame.
    
    Args:
        polygons_gdf (GeoDataFrame): GeoDataFrame containing polygons.
        roads_gdf (GeoDataFrame): GeoDataFrame containing roads.
        polygon_column_name (str): Name of the column in polygons_gdf to use for grouping.

    Returns:
        GeoDataFrame: Original polygons GeoDataFrame with a "road_count" column added.
    """
    
    # spatial join
    joined = gpd.sjoin(polygons_gdf, roads_gdf, how='left', op='intersects')
    
    # Group by the polygon column and count the number of roads in each
    road_counts = joined.groupby(polygon_column_name).size().reset_index(name='road_count')
    
    # Merge the road counts back into the polygons GeoDataFrame
    polygons_gdf = polygons_gdf.merge(road_counts, on=polygon_column_name, how='left')

     # Calculate road density (area divided by road_count). It is mulitiplied by 10000 for ease of understanding the numbers involved with this
    polygons_gdf['road_density'] = (polygons_gdf['road_count'] / polygons_gdf['area'] ) * 10000
    
    return polygons_gdf

neighbourhoods = count_roads_within_polygons(neighbourhoods, os_open_roads_clip, 'geometry')

# Drop rows with road_density below 0.2 or less than 4 roads
neighbourhoods = neighbourhoods[(neighbourhoods['road_count'] > 2)]
neighbourhoods = neighbourhoods[(neighbourhoods['road_density'] > 0.2)]

c:\Users\b8008458\Anaconda3\envs\ox_151\Lib\site-packages\IPython\core\interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [250]:
neighbourhoods.explore()